In [ ]:
import openpyxl as px
from openpyxl import Workbook
from openpyxl import load_workbook
import numpy as np
import urllib
from urllib.request import urlopen
import os
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
import time
import math as ma
import threading as thr
import multiprocessing as mp
import requests

In [ ]:
friend_id_map={} # Write the name of your friends and the value as their cf id

In [5]:
def table_maker(ratings):
    page=1
    
    #### Calculating the total number of pages
    base_url=f"https://codeforces.com/problemset/page/{page}?tags={ratings[0]}-{ratings[0]}"
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    request=urllib.request.Request(base_url,None,headers)
    response = urllib.request.urlopen(request)
    base_datab = response.readlines()
    base_data=[''.join(str(c)) for c in base_datab]
    for i in range(len(base_data)):
        if '<ul>' in base_data[i]:
            pg_start=i
    for i in range(len(base_data)):
        if '</ul>' in base_data[i]:
            pg_end=i
    line_pages=[i for i in range(pg_start+3,pg_end,3)]
    pages=len([int(base_data[i][-17:-16]) for i in line_pages])
    del base_url,user_agent,headers,request,response,base_datab,base_data,pg_start,pg_end,line_pages

    #### Processing
    for page in range(1,pages+1):
        """ Always specify the Rating as a list with two values."""
        #### Defining URL
        url = f"https://codeforces.com/problemset/page/{page}?tags={ratings[0]}-{ratings[0]}"

        #### Accessing url
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,}
        request=urllib.request.Request(url,None,headers)
        response = urllib.request.urlopen(request)
        datab = response.readlines()
        data=[''.join(str(c)) for c in datab]
        
        #### All junk values needed later
        
        ### Calculating the span of the data
        for i in range(len(data)):
            if "<table" in data[i]:
                t_start=i
                break
        for i in range(len(data)):
            if "</table>" in data[i]:
                t_end=i
                break
        
        ### The strings needed for matching
        x_id='<a href="/problemset/problem/'
        x_tag_start='<div style="float: right; font-size: 1.1rem; padding-top: 1px; text-align: right;">'
        x_tag_end='<span class="act-item">\\r\\n'
        x_solved="<a title=\"Participants solved the problem\""
        x_rating='<span title="Difficulty"class="ProblemRating">'
        domain="https://codeforces.com"
        
        #### Listing out all the data
        
        ### Problem ID, Names and Ratings
        temp_line_id=[]
        for i in range(t_start,t_end):
            if(x_id in data[i]):
                temp_line_id.append(i)
        line_rating=[]
        for i in range(t_start,t_end):
            if(x_rating in data[i]):
                line_rating.append(i)
        line_id=[]
        line_name=[]
        line_name=list(np.array(temp_line_id[1::2])+1)
        line_id=temp_line_id[0::2]
        problem_name=[data[i][2:-9].strip() for i in line_name]
        problem_id=[data[i][47:-7] for i in line_id]
        problem_id=[i.replace("/","") for i in problem_id]
        problem_url=[domain+"/"+data[i][28:-7] for i in line_id]
        problem_rating=[int(data[i][68:-12]) for i in line_rating]
        samples=len(problem_id)
        
        ### Problem Tags
        tag_start=[]
        tag_end=[]
        for i in range(t_start,t_end):
            if x_tag_start in data[i]:
                tag_start.append(i+1)
            elif x_tag_end in data[i]:
                tag_end.append(i-3)
            else:
                continue
        tags_lines=np.array([[tag_start[i],tag_end[i]] for i in range(samples)])
        all_tags=[]
        for i in range(samples):
            tags=[]
            for j in range(tags_lines[i][0],tags_lines[i][1]):
                tags.append(data[j].split(">")[1][:-3])
            all_tags.append(tags)
        lis_tags=[", ".join(i) for i in all_tags]
        
        ### Location for saving files/sheets
        
        if ratings[0]==ratings[1]:
            save_path=f"{ratings[0]}/{ratings[0]}.xlsx"
            sheet_name=f'{ratings[0]}_pg{page}'
            print(page)
            dir=f"{ratings[0]}"
        else:
            save_path=f"{ratings[0]}-{ratings[1]}/{ratings[0]}.xlsx"
            sheet_name=f'{ratings[0]}-{ratings[1]}_pg{page}'
            dir=f"{ratings[0]}-{ratings[1]}"
        
        ### Solved Numbers
        solved_lines=[]
        for i in range(t_start,t_end):
            if x_solved in data[i]:
                solved_lines.append(i)
        solved_url=[]
        for i in solved_lines:
            solved_url.append(domain+data[i][71:].split('"')[0])
        solved_numbers=[]
        for i in solved_lines:
            solved_numbers.append(int(data[i][150:].split('>')[1][7:-3]))
        
        ### Working with the excel sheets  
        try:
            wb=load_workbook(save_path)
            if sheet_name in wb.sheetnames:
                break
            else:
                wb.create_sheet(sheet_name)    
        except FileNotFoundError:
            wb=Workbook()
            wb.create_sheet(sheet_name)
            wb.remove_sheet(wb['Sheet'])                 
        wb.active=wb[sheet_name]
        sheet=wb.active
        sheet["A1"] = "ID"
        sheet["B1"] = "Name"
        sheet["C1"] = "Tags"
        sheet["D1"] = "Rating"
        sheet["E1"] = "Solved numbers"
        
        ### Loading and formatting the data in excel
        for i in range(samples):
            #Problem ID
            sheet.cell(row=2+i, column=1).value='=HYPERLINK("{}", "{}")'.format(problem_url[i],problem_id[i])
            sheet.cell(row=2+i, column=1).font=px.styles.Font(color="0000FF")
            #Problem Name
            sheet.cell(row=2+i, column=2).value='=HYPERLINK("{}", "{}")'.format(problem_url[i],problem_name[i])
            sheet.cell(row=2+i, column=2).font=px.styles.Font(color="0000FF")
            #Problem Tags
            sheet.cell(row=2+i, column=3).value=lis_tags[i]
            #Problem Rating
            sheet.cell(row=2+i, column=4).value=problem_rating[i]
            #Problem Solved Numbers
            # sheet.cell(row=2+i, column=5).value='=HYPERLINK("{}", "{}")'.format(solved_url[i],solved_numbers[i])
            sheet.cell(row=2+i, column=5).value='=HYPERLINK("{}", {})'.format(solved_url[i],solved_numbers[i])
            sheet.cell(row=2+i, column=5).font=px.styles.Font(color="0000FF")

        #### Final saving
        try:  
            wb.save(save_path)
            print("File ",save_path," was successful.")
            pages+=1
        except Exception as e:
            os.mkdir(dir) 
            wb.save(save_path)
            print("File ",save_path," was successful. but directory : ",dir," didn't exist so we created it.")
            pages+=1

In [ ]:
all_ratings=[1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200]
start=time.time()
for i in all_ratings:
    table_maker([i,i])
end=time.time()
time_taken=end-start
print(time_taken)